# Precipitation Variability Across Timescales

This notebook demonstrates how to use the precipitation variability metrics driver. 

This notebook should be run in an environment with python, jupyterlab, pcmdi metrics package, and cdat installed. It is expected that you have downloaded the sample data as demonstrated in [the download notebook](Demo_0_download_data.ipynb).  

The following cell reads in the choices you made during the download data step:

In [1]:
from user_choices import demo_data_directory, demo_output_directory

## Basic Use

Use the `--help` flag for assistance with the precip variability driver:

In [2]:
%%bash
variability_across_timescales_PS_driver.py --help

usage: variability_across_timescales_PS_driver.py [-h]
                                                  [--parameters PARAMETERS]
                                                  [--diags OTHER_PARAMETERS [OTHER_PARAMETERS ...]]
                                                  [--mip MIP] [--mod MOD]
                                                  [--var VAR] [--frq FRQ]
                                                  [--modpath MODPATH]
                                                  [--outdir OUTDIR]
                                                  [--prd PRD] [--fac FAC]
                                                  [--nperseg NPERSEG]
                                                  [--noverlap NOVERLAP]

optional arguments:
  -h, --help            show this help message and exit
  --parameters PARAMETERS, -p PARAMETERS
  --diags OTHER_PARAMETERS [OTHER_PARAMETERS ...], -d OTHER_PARAMETERS [OTHER_PARAMETERS ...]
                        Path to other user-defined para

Settings are specified in a separate parameter file, which is printed here:

In [3]:
# print parameter file
with open("basic_precip_variability_param.py") as f:
    print(f.read())

mip = "cmip5"
exp = "historical"
mod = "GISS-E2-H"
var = "pr"
frq = "day"
modpath = 'demo_data_standard/CMIP5_demo_precip_var/'
outdir = 'demo_output/precip_variability/'
prd = [2000,2001]  # analysis period
fac = 86400  # factor to make unit of [mm/day]

# length of segment in power spectra (~10 years)
# shortened to 2 years for demo purposes
nperseg = 2 * 365
# length of overlap between segments in power spectra (~5 years)
# shortened to 1 year for demo purposes
noverlap = 1 * 365



The parameter file is passed to the driver using the `-p` flag, similar to other PMP metrics. The next cell runs the driver as a subprocess:

In [9]:
%%bash
variability_across_timescales_PS_driver.py -p basic_precip_variability_param.py

demo_data_standard/CMIP5_demo_precip_var/
GISS-E2-H
[2000, 2001]
730 365
# of data: 1
['GISS-E2-H.r6i1p1']
GISS-E2-H.r6i1p1 365_day
Complete regridding from (365, 90, 144) to (365, 90, 180)
2000 (365, 90, 180)
Complete regridding from (365, 90, 144) to (365, 90, 180)
2001 (730, 90, 180)
Complete calculating climatology and anomaly for calendar of 365_day
Complete power spectra (segment:  730  nps: 1.0 )
Complete domain and frequency average of spectral power
Complete power spectra (segment:  730  nps: 1.0 )
Complete domain and frequency average of spectral power


Running the precipitation variability driver produces three output files, found in the demo output directory:
Spatial pattern of spectral power (forced and unforced variability)

In [10]:
!ls {demo_output_directory + "/precip_variability"}

PS_pr.day_regrid.180x90_GISS-E2-H.r6i1p1.nc
PS_pr.day_regrid.180x90_GISS-E2-H.r6i1p1_unforced.nc
PS_pr.day_regrid.180x90_area.freq.mean_GISS-E2-H.r6i1p1.json


Metrics results are found in the JSON output.

In [7]:
import json
import os
output_path = os.path.join(demo_output_directory,"precip_variability/PS_pr.day_regrid.180x90_area.freq.mean_GISS-E2-H.r6i1p1.json")
with open(output_path) as f:
    metric = json.load(f)
print(json.dumps(metric, indent=2))

{
  "forced": {
    "cmip5": {
      "GISS-E2-H.r6i1p1": {
        "power": {
          "Land_30N50N": {
            "annual": 1.1587082092242942,
            "semi-annual": 0.41744881229451547
          },
          "Land_30S30N": {
            "annual": 6.971767234863867,
            "semi-annual": 1.1463392061273523
          },
          "Land_50S30S": {
            "annual": 0.6081871639608728,
            "semi-annual": 0.30708232514267686
          },
          "Land_50S50N": {
            "annual": 4.864266970917859,
            "semi-annual": 0.880099077324579
          },
          "Ocean_30N50N": {
            "annual": 1.573720339534381,
            "semi-annual": 0.31178472575620936
          },
          "Ocean_30S30N": {
            "annual": 4.868478624745868,
            "semi-annual": 1.485758806678225
          },
          "Ocean_50S30S": {
            "annual": 0.548059204349184,
            "semi-annual": 0.18055034836612316
          },
          "Ocean_50S50N": 

In [ ]:
%%bash
variability_across_timescales_PS_driver.py -p basic_precip_variability_param.py \
